[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain Academy

Добро пожаловать в LangChain Academy!

## Контекст

В LangChain наша цель — упростить создание приложений на базе LLM (Language Models). Один из типов приложений, которые можно создавать, — это агенты. Существует большой интерес к созданию агентов, потому что они могут автоматизировать широкий спектр задач, которые раньше были невозможны.

Однако на практике создание систем, которые надежно выполняют эти задачи, чрезвычайно сложно. Работая с пользователями и внедряя агентов в продукцию, мы поняли, что часто требуется больше контроля. Возможно, вам нужно, чтобы агент всегда сначала вызывал конкретный инструмент или использовал различные запросы в зависимости от своего состояния.

Для решения этой проблемы мы создали [LangGraph](https://langchain-ai.github.io/langgraph/) — фреймворк для создания агентов и многосоставных агентов. Отдельный от пакета LangChain, основная философия дизайна LangGraph заключается в том, чтобы помочь разработчикам добавить точность и контроль в рабочие процессы агентов, подходящие для сложности реальных систем.

## Структура курса

Курс структурирован как набор модулей, каждый из которых сосредоточен на определенной теме, связанной с LangGraph. Вы увидите папку для каждого модуля, которая содержит серию ноутбуков. К каждому ноутбуку будет прилагаться видео, которое поможет разобраться с концепциями, но ноутбуки также могут быть самостоятельными, то есть они содержат объяснения и могут быть просмотрены независимо от видео. Каждая папка модуля также содержит папку `studio`, которая содержит набор графиков, которые можно загрузить в [LangGraph Studio](https://github.com/langchain-ai/langgraph-studio), нашу среду разработки для создания приложений на LangGraph.

## Настройка

Прежде чем начать, пожалуйста, следуйте инструкциям в `README`, чтобы создать среду и установить зависимости.

## Модели чатов

В этом курсе мы будем использовать [Модели чатов](https://python.langchain.com/v0.2/docs/concepts/#chat-models), которые выполняют несколько функций: принимают последовательность сообщений в качестве входных данных и возвращают сообщения чатов в качестве выходных. LangChain не хостит модели чатов, вместо этого мы полагаемся на сторонние интеграции. [Здесь](https://python.langchain.com/v0.2/docs/integrations/chat/) представлен список сторонних интеграций моделей чатов в LangChain! По умолчанию курс будет использовать [ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/), так как она и популярна, и производительна. Как указано, пожалуйста, убедитесь, что у вас есть `OPENAI_API_KEY`.

Давайте проверим, установлен ли ваш `OPENAI_API_KEY`, и если нет, вам будет предложено ввести его.

In [1]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

[Здесь](https://python.langchain.com/v0.2/docs/how_to/#chat-models) представлен полезный гайд по всем возможностям работы с моделями чатов, но ниже мы покажем несколько основных моментов. Если вы выполнили команду `pip install -r requirements.txt`, как указано в README, то вы установили пакет `langchain-openai`. С помощью этого пакета мы можем создать объект модели `ChatOpenAI`. Если вы только что зарегистрировались для использования API, вам должны предоставить [бесплатные кредиты](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517), которые можно применить к любой из моделей. Цены на различные модели можно посмотреть [здесь](https://openai.com/api/pricing/). Ноутбуки по умолчанию будут использовать модель `gpt-4o`, так как она представляет собой хорошее соотношение качества, цены и скорости [подробнее здесь](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini), но вы также можете выбрать модели из серии `gpt-3.5`, которые стоят дешевле.

С моделями чатов можно настроить [несколько стандартных параметров](https://python.langchain.com/v0.2/docs/concepts/#chat-models). Два из самых популярных параметров:

* `model`: название модели
* `temperature`: температура выборки

`Temperature` (температура) управляет случайностью или креативностью вывода модели, где низкая температура (близкая к 0) означает более детерминированные и сфокусированные результаты. Это хорошо для задач, требующих точности или фактических ответов. Высокая температура (близкая к 1) подходит для творческих задач или генерации разнообразных ответов.

In [3]:
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

Модели чатов в LangChain имеют ряд [методов по умолчанию](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). В основном мы будем использовать:

* `stream`: передавать части ответа по мере их получения
* `invoke`: вызывать цепочку для входных данных

Как уже упоминалось, модели чатов принимают [сообщения](https://python.langchain.com/v0.2/docs/concepts/#messages) в качестве входных данных. Сообщения имеют роль (которая описывает, кто говорит это сообщение) и свойство содержимого. Мы подробнее обсудим это позже, но сейчас давайте покажем основные моменты.

In [4]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
gpt4o_chat.invoke(messages)

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f785eb5f47', 'finish_reason': 'stop', 'logprobs': None}, id='run-c956e8a1-1689-43ef-9bc4-9bbed89620b4-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Мы получаем ответ типа `AIMessage`. Также обратите внимание, что модель можно вызвать с помощью строки. Когда строка передается в качестве входных данных, она преобразуется в `HumanMessage` и затем передается в основную модель.


In [5]:
gpt4o_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_5f20662549', 'finish_reason': 'stop', 'logprobs': None}, id='run-6f1ebb07-41f3-492d-80a7-88bbe07eae79-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
gpt35_chat.invoke("hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 9, 'total_tokens': 19, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8f2a7cbf-5714-42c6-83f4-59cd244fb8de-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Интерфейс одинаков для всех моделей чатов, и модели обычно инициализируются один раз в начале каждого ноутбука.

Таким образом, вы можете легко переключаться между моделями, не меняя код, если у вас есть предпочтение к другому поставщику.

## Инструменты поиска

Вы также увидите [Tavily](https://tavily.com/) в README, это поисковая система, оптимизированная для LLM и RAG, направленная на эффективные, быстрые и постоянные результаты поиска. Как уже упоминалось, легко зарегистрироваться и воспользоваться щедрым бесплатным тарифом. Некоторые уроки (в Модуле 4) по умолчанию будут использовать Tavily, но, конечно, можно использовать и другие инструменты поиска, если вы хотите изменить код для себя.

In [9]:
_set_env("TAVILY_API_KEY")

In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily_search = TavilySearchResults(max_results=3)
search_docs = tavily_search.invoke("What is LangGraph?")

In [11]:
search_docs

[{'url': 'https://langchain-ai.github.io/langgraph/',
  'content': 'Overview¶. LangGraph is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows. Compared to other LLM frameworks, it offers these core benefits: cycles, controllability, and persistence. LangGraph allows you to define flows that involve cycles, essential for most agentic architectures, differentiating it from DAG-based solutions.'},
 {'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
  'content': 'LangGraph is a library within the LangChain ecosystem designed to tackle these challenges head-on. LangGraph provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured manner.'},
 {'url': 'https://medium.com/@cplog/introduction-to-langgraph-a-beginners-guide-14f9be027141',
  'content': 'LangGraph is a versatile tool for building complex, stateful applications with LLMs. By understanding its core concep